# Lab 5.4: Chi-Square Tests

## Outline

- Chi-square test for goodness of fit
- Chi-square test for independence

In [4]:
%pylab inline

import pandas as pd
import statsmodels.api as sm

from IPython.display import Latex

from scipy import stats
from statsmodels.graphics.gofplots import qqplot
from statsmodels.distributions.empirical_distribution import ECDF

Populating the interactive namespace from numpy and matplotlib


**Questions 1**  

Seven percent of mutual fund investors rate corporate stocks “very safe,” 58% rate them “somewhat safe,” 24% rate them “not very safe,” 4% rate them “not at all safe,” and 7% are “not sure.” A BusinessWeek/Harris poll asked 529 mutual fund investors how they would rate corporate bonds on safety. The responses are as follows.  

<img src="images/q1.png" width="300">  

Do mutual fund investors’ attitudes toward corporate bonds differ from their attitudes toward
corporate stocks? Clearly state the null and alternative hypotheses.

#### Answer 1

$H_0$: the stocks and bonds attitudes differ  
$H_a$: the stocks and bonds attitudes are similar  

In [13]:
ratings = ["Very safe", "Somewhat safe", "Not very safe", "Not at all safe", "Not sure"]
stocks_bonds = pd.DataFrame({"Stocks": [.07, .58, .24, .04, .07], 
                             "Bonds": [48/529, 323/529, 79/529, 16/529, 63/529]}, index=ratings).T
chi2, p, dof, expected = stats.chi2_contingency(stocks_bonds)

In [6]:
Latex(r"$\chi^2 = {:.4}; p = {:.2}$".format(chi2, p))

<IPython.core.display.Latex object>

In [14]:
expected

array([[ 0.08036862,  0.59529301,  0.19466919,  0.03512287,  0.09454631],
       [ 0.08036862,  0.59529301,  0.19466919,  0.03512287,  0.09454631]])

#### Answer 1 Commentary

The p-value is 1, so that means that we fail to reject that the attitudes differ. The evidence shows that stock and bond attitudes are actually quite dependent, but we can't prove that for sure.

**Question 2**  

A news article reports that "Americans have differing views on two potentially inconvenient and invasive practices that airports could implement to uncover potential terrorist attacks." This news piece was based on a survey conducted among a random sample of 1,137 adults nationwide, interviewed by telephone November 7-10, 2010, where one of the questions on the survey was "Some airports are now using 'full-body' digital x-ray machines to electronically screen passengers in airport security lines. Do you think these new x-ray machines should or should not be used at airports?" Below is a summary of responses based on party affiliation.  

<img src="images/q4.png" width="550">  

The differences in each political group may be due to chance. Complete the following computations under the null
hypothesis of independence between an individual's party affiliation and his/her support of full-body scans. It may be useful to first add on an extra column for row totals before proceeding with the computations.  

1) How many Republicans would you expect to not support the use of full-body scans?  

2) How many Democrats would you expect to support the use of full-body scans?  

3) How many Independents would you expect to not know or not answer?  

4) Test if an individual's party affiliation affects his/her support of full-body scans. Clearly state the null and alternative hypotheses in your test.

### Answer 2

In [32]:
party = ["Republican", "Democrat", "Independent", "Total"]
body_scans = pd.DataFrame({"Should": [264, 299, 351, (264+299+351)], 
                           "Don't know/No Answer": [16, 15, 22, (16+15+22)], 
                           "Should not": [38, 55, 77, (38+55+77)], 
                           "Total": [318, 369, 450, (318+369+450)]},index=party).T
chi2, p, dof, expected_party = stats.chi2_contingency(body_scans)

In [33]:
body_scans

,Republican,Democrat,Independent,Total
Don't know/No Answer,16,15,22,53
Should,264,299,351,914
Should not,38,55,77,170
Total,318,369,450,1137


In [34]:
expected_party

array([[   14.823219  ,    17.2005277 ,    20.9762533 ,    53.        ],
       [  255.63060686,   296.62796834,   361.7414248 ,   914.        ],
       [   47.54617414,    55.17150396,    67.2823219 ,   170.        ],
       [  318.        ,   369.        ,   450.        ,  1137.        ]])

#### Answer 2.1

1) How many Republicans would you expect to not support the use of full-body scans?  


In [38]:
print("We would expect", round(expected_party[2][0], 3), "Republicans to not support body scans")

We would expect 47.546 Republicans to not support body scans


**Question 3**  

A clothes retailer believes that there is no difference in sales across Monday, Tuesday and Wednesday. You are given the data in a `cloth_sales` table (in RDS) to test the claim. The table contains two columns: `dt` for the date, and `sales`, containing the count of sales for that day. Start by drawing up the table for the observed and expected frequencies for the chi-square test. 

**Hint:** 
- It will probably be easiest to extract the week of the year (`week`) and the day of the week (`dow`) using the [`date_trunc`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC) and [`date_part`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) functions in PostgreSQL respectively. You can then [`pivot`](http://pandas.pydata.org/pandas-docs/stable/reshaping.html#reshaping-by-pivoting-dataframe-objects) the table with pandas. The `head` of the resulting data frame should look something like:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>date_part</th>
      <th>1.0</th>
      <th>2.0</th>
      <th>3.0</th>
    </tr>
    <tr>
      <th>date_trunc</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2014-10-27</th>
      <td>1016</td>
      <td>978</td>
      <td>1010</td>
    </tr>
    <tr>
      <th>2014-11-03</th>
      <td>987</td>
      <td>991</td>
      <td>997</td>
    </tr>
    <tr>
      <th>2014-11-10</th>
      <td>1014</td>
      <td>983</td>
      <td>1002</td>
    </tr>
    <tr>
      <th>2014-11-17</th>
      <td>991</td>
      <td>945</td>
      <td>992</td>
    </tr>
    <tr>
      <th>2014-11-24</th>
      <td>1001</td>
      <td>1058</td>
      <td>1002</td>
    </tr>
  </tbody>
</table>  

(N.B. `date_part('dow', dt)` will return the number of days after Sunday, so Monday = 1.0, Tuesday = 2.0, and so on.)
   
- Use `scipy.stats.chisquare()` to carry out a goodness of fit test

**Question 4**  

1) A law suit has been filed against a university for a charge of sexual discrimination against female applicants during the admissions process. Use the data below to test whether sex affects admission at this university.
      
   **Hint:**
   - Construct your null and alternative hypotheses
   - Use `scipy.stats.chi2_contingency()` to carry out a test for independence
   - The function takes the contingency table as an `numpy` array as the first argument


|        | Admitted | Not Admitted |
|--------|----------|--------------|
| Male   | 3715     | 4727         |
| Female | 1513     | 2808         |

2) You are also given the breakdown of the female and male admissions by department (A to F).

<img src="images/paradox_1.png" width="300px">

Test if sex and department are independent in terms of number of applicants.

3) (Extra Credit) Based on all the data you are given, is it fair to say that there is sexual discirmination in the admission process? Explain your answer. (Hint: Simpson's paradox)

#### Hint

Some functions that may be useful to you:

- From the `sqlalchemy` package:
    - `create_engine`
- From the `pandas` package:
    - `read_sql`
    - `pivot`
- From the `scipy.stats` package:
    - `chisquare`
    - `chi2_contingency`